# Baseball Predictions

## OG algo

In [1]:
#!pip install pandas
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.model_selection import train_test_split

In [4]:
#grab data
data_path = 'og_algo/train.csv'
data = pd.read_csv(data_path)

testing_path = 'og_algo/data21.csv'
testing = pd.read_csv(testing_path)

#print
data.describe()

,m1 h,m2 h,m3 h,m4 h,m1 a,m2 a,m3 a,m4 a,winner
count,2763.000000,2763.000000,2763.000000,2763.000000,2763.000000,2763.000000,2763.000000,2763.000000,2763.000000
mean,0.377351,0.042599,-0.125212,0.160548,0.384202,0.020174,-0.103820,0.228418,0.534202
std,1.541160,0.848823,1.228167,2.527689,1.738104,0.848673,1.194767,2.366799,0.498919
min,-25.770000,-2.390000,-6.251034,-83.900000,-48.690000,-2.310000,-6.368462,-84.300000,0.000000
25%,-0.220000,-0.560000,-0.730974,-0.638366,-0.170000,-0.600000,-0.719985,-0.564976,0.000000
50%,0.530000,0.130000,-0.003274,0.427595,0.550000,0.040000,0.000000,0.401232,1.000000
75%,1.160000,0.650000,0.489338,1.308438,1.160000,0.650000,0.531609,1.315699,1.000000
max,5.920000,1.920000,24.879934,5.920000,5.960000,1.760000,9.921471,6.100000,1.000000


In [6]:
#y is the value we are trying to predict
train_y = data.winner
#these are the variables we are using to make the prediction
features = ['m1 h', 'm2 h', 'm3 h', 'm4 h', 'm1 a', 'm2 a', 'm3 a', 'm4 a']
train_X = data[features]

val_y = testing.winner
val_X = testing[features]

In [11]:
#train_X, val_X, train_y, val_y = train_test_split(x, y, random_state = 1, test_size = 0.3)

data_model = DecisionTreeRegressor(random_state=1)
data_model.fit(train_X, train_y)

# get predicted prices on validation data
val_predictions = data_model.predict(val_X)
print("Decision Tree Regressor: ")
print(1 - mean_absolute_error(val_y, val_predictions))

Decision Tree Regressor: 
0.5132939438700148


In [10]:
random_model = RandomForestRegressor(random_state=1)
random_model.fit(train_X, train_y)

forest_preds = random_model.predict(val_X)
print("Random Forest Regression: ")
print(1 - mean_absolute_error(val_y, forest_preds))

Random Forest Regression: 
0.5146750369276218


In [12]:
from sklearn.ensemble import GradientBoostingClassifier
class_model = GradientBoostingClassifier(random_state=1)
class_model.fit(train_X, train_y)

class_preds = class_model.predict(val_X)
print("Gradient Boosting Classifer: ")
print(accuracy_score(val_y, class_preds))

Gradient Boosting Classifer: 
0.5664697193500738


In [13]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

lr.fit(train_X, train_y)
lr_preds = lr.predict(val_X)
#print(mean_absolute_error(val_y, lr_preds))
print("Logistic Regression: ")
print(accuracy_score(val_y, lr_preds))

Logistic Regression: 
0.5664697193500738


In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(StandardScaler(),
                     LogisticRegression())

pipe.fit(train_X, train_y)
p_preds = pipe.predict(val_X)

print("Logistic Regression w/ Pipeline: ")
print(accuracy_score(val_y, p_preds))

Logistic Regression w/ Pipeline: 
0.5664697193500738


## Raw Data

In [7]:
data_path = 'raw/train.csv'
data = pd.read_csv(data_path)

testing_path = 'raw/data21.csv'
testing = pd.read_csv(testing_path)

data.describe()

,hrs,hra,hsera,hr,hera,ars,ara,asera,ar,aera,winner
count,2704.000000,2704.000000,2704.000000,2704.000000,2704.000000,2704.000000,2704.000000,2704.000000,2704.000000,2704.000000,2704.000000
mean,4.628702,4.611864,4.463825,1.077563,4.245936,4.617489,4.620477,4.410049,1.070877,4.248706,0.534763
std,0.544129,0.609479,2.617819,0.398228,1.600610,0.543556,0.605979,2.447662,0.392274,1.812099,0.498882
min,3.410000,3.040000,0.000000,0.369231,0.000000,3.430000,3.040000,0.000000,0.375000,0.000000,0.000000
25%,4.280000,4.140000,3.240997,0.786831,3.390000,4.280000,4.140000,3.255319,0.785440,3.400000,0.000000
50%,4.620000,4.570000,4.157973,1.043478,4.090000,4.590000,4.600000,4.180872,1.038462,4.070000,1.000000
75%,5.010000,5.070000,5.311840,1.281250,4.840000,5.010000,5.080000,5.284698,1.279559,4.820000,1.000000
max,6.030000,6.450000,90.000000,2.416667,30.860000,5.990000,6.420000,90.000000,2.457143,54.000000,1.000000


In [8]:
#y is the value we are trying to predict
train_y = data.winner
#these are the variables we are using to make the prediction
features = ['hrs', 'hra', 'hsera', 'hr', 'hera', 'ars', 'ara', 'asera', 'ar', 'aera']
train_X = data[features]

val_y = testing.winner
val_X = testing[features]

In [9]:
data_model = DecisionTreeRegressor(random_state=1)
data_model.fit(train_X, train_y)

# get predicted prices on validation data
val_predictions = data_model.predict(val_X)
print("Decision Tree Regressor: ")
print(1 - mean_absolute_error(val_y, val_predictions))

Decision Tree Regressor: 
0.5281743050338092


In [10]:
random_model = RandomForestRegressor(random_state=1)
random_model.fit(train_X, train_y)

forest_preds = random_model.predict(val_X)
print("Random Forest Regression: ")
print(1 - mean_absolute_error(val_y, forest_preds))

Random Forest Regression: 
0.5180540946656649


In [11]:
from sklearn.ensemble import GradientBoostingClassifier
class_model = GradientBoostingClassifier(random_state=1)
class_model.fit(train_X, train_y)

class_preds = class_model.predict(val_X)
print("Gradient Boosting Classifer: ")
print(accuracy_score(val_y, class_preds))

Gradient Boosting Classifer: 
0.5424492862509391


In [12]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

lr.fit(train_X, train_y)
lr_preds = lr.predict(val_X)
#print(mean_absolute_error(val_y, lr_preds))
print("Logistic Regression: ")
print(accuracy_score(val_y, lr_preds))

Logistic Regression: 
0.5732531930879038


In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(StandardScaler(),
                     LogisticRegression())

pipe.fit(train_X, train_y)
p_preds = pipe.predict(val_X)

print("Logistic Regression w/ Pipeline: ")
print(accuracy_score(val_y, p_preds))

Logistic Regression w/ Pipeline: 
0.574755822689707
